In [6]:
(asdf:load-system :imago/jupyter)
(asdf:load-system :diff-boundary-corrfn)
(asdf:load-system :py4cl)
(asdf:load-system :array-operations)

T

T

T

T

In [2]:
(load
  (asdf:system-relative-pathname :diff-boundary-corrfn
                                 #p"examples/functions.lisp"))

T

In [16]:
;; These parameters make calculations faster
(setq diff-boundary-corrfn:*lattice-elements* 800
      diff-boundary-corrfn:*ε-threshold* 1d-3)

0.001d0

In [19]:
;; Calculate interface for 3D ball with radius R=0.2
(defparameter *interface* 
    (diff-boundary-corrfn:interface #'example-functions:ball 3 2d-1))

*INTERFACE*

In [7]:
(defparameter *sss*
  (sb-int:with-float-traps-masked
    (:divide-by-zero)
    (loop for x from 2d-2 to 4.1d-1 by 5d-3 collect
          (loop for y from 2d-2 to 4.1d-1 by 5d-3 collect
                (diff-boundary-corrfn:surface3 *interface* (list x 0d0 0d0) (list 0d0 y 0d0))))))

*SSS*

In [20]:
;; Convert F_{sss} to an array and set the upper bound to 60
(defparameter *array*
  (let* ((length (length *sss*))
         (array (make-array (list length length)
                            :element-type 'double-float
                            :initial-contents *sss*)))
    (aops:vectorize* 'double-float
        (array)
      (min array 60d0))))

*ARRAY*

In [13]:
(py4cl:import-module "matplotlib.pyplot" :as "plt")
(py4cl:import-module "numpy" :as "np")
(py4cl:import-module "scipy.interpolate" :as "inter")
(py4cl:import-function "slice")

T

T

T

SLICE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining PLT:ARROW in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining PLT:AXES in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining PLT:FIGURE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining PLT:SUBPLOT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining PLT:TEXT in DEFUN


In [21]:
;; Plot F_{sss} function with arguments (x, 0, 0) and (0, y, 0)
;; x and y vary from 0.02 to 4.1 by 0.005
(let* ((x (np:linspace 0 0.41 (length *sss*)))
       (grid (np:meshgrid x x))
       (ax (plt:axes '(0 0 1 1) :projection "3d")))
  (py4cl:chain ax (plot_surface (elt grid 0) (elt grid 1) *array*))
  (plt:show))

NIL